In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
id_gpu = '2'
n_users = 70
os.environ['CUDA_VISIBLE_DEVICES'] = id_gpu

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from datasets import load_dataset

dataset = load_dataset("LeoTungAnh/kdd210_hourly")
dataset

DatasetDict({
    train: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
    validation: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
    test: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
})

In [6]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [7]:
len(test_dataset[0]['target'])

10898

In [8]:
type(test_dataset)

datasets.arrow_dataset.Dataset

In [9]:
import pandas as pd

# Create a list of dictionaries, where each dictionary represents a row in the DataFrame
df = pd.DataFrame()

for row in test_dataset:
  df[row['item_id']] = row['target']

df['date'] = '2015-01-01 00:00:00'
df = df.set_index("date")
df.head()

/tmp/ipykernel_4085416/2019493983.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[row['item_id']] = row['target']
/tmp/ipykernel_4085416/2019493983.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[row['item_id']] = row['target']
/tmp/ipykernel_4085416/2019493983.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T201,T202,T203,T204,T205,T206,T207,T208,T209,T210
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,1.468122,1.538769,-0.030617,-0.781501,-0.802695,-0.772417,-0.378809,-0.237514,-0.393948,-0.810264,...,-0.131542,-0.785538,-0.807741,-0.777464,1.493353,1.654833,-0.025571,-0.776959,-0.777464,-0.792602
2015-01-01 00:00:00,1.316855,1.449373,-0.079674,-0.773862,-0.798327,-0.767746,-0.512904,0.000000,-0.507808,-0.803933,...,-0.140836,-0.775391,-0.798327,-0.772842,1.444276,1.724601,-0.023609,-0.804443,-0.767746,-0.783036
2015-01-01 00:00:00,1.261700,1.636722,-0.061296,-0.759776,-0.780089,-0.754046,-0.639456,-0.582161,-0.597787,-0.792069,...,-0.134217,-0.759776,-0.780089,-0.754046,1.501297,1.772147,-0.009210,-0.791548,-0.748838,-0.769672
2015-01-01 00:00:00,1.487590,1.831615,-0.022891,-0.730291,-0.753942,-0.721690,-0.646435,-0.560429,-0.555053,-0.765768,...,-0.114272,-0.733516,-0.753942,-0.732441,0.525398,0.000000,-0.270158,-0.744267,-0.678687,-0.748567
2015-01-01 00:00:00,1.842409,2.225288,0.150884,-0.702879,-0.723452,-0.694879,-0.632018,-0.500582,-0.454865,-0.741738,...,-0.066271,-0.707451,-0.729166,-0.717737,-0.672020,0.000000,-0.569157,-0.743453,-0.557728,-0.723452


In [10]:
# # Save the DataFrame to a CSV file
# df.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/kdd/kdd.csv', index_label='date')

In [11]:
df.shape

(10898, 210)

In [12]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(df.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = id_gpu
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path = './dataset/kdd/'
args.data_path = 'kdd.csv'
args.model_id = 'kdd_96_96'
args.data_path

'kdd.csv'

In [13]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [14]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, df.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, df.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [15]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(3, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(3, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
            

In [16]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on KDD dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on KDD dataset",
    # Hyperparams
    config={
        "dataset": "KDD210",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 210,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [17]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 122.97710919380188
Epoch: 2 cost time: 126.52398824691772
Epoch: 3 cost time: 126.11972498893738
Epoch: 4 cost time: 133.78196167945862
Epoch: 5 cost time: 134.90438890457153
Epoch: 1 cost time: 135.96238613128662
Epoch: 2 cost time: 135.7364478111267
Epoch: 3 cost time: 136.03667879104614
Epoch: 4 cost time: 135.88181376457214
Epoch: 5 cost time: 136.3942425251007
Epoch: 1 cost time: 137.14592003822327
Epoch: 2 cost time: 135.73837733268738
Epoch: 3 cost time: 135.67568182945251
Epoch: 4 cost time: 136.1249635219574
Epoch: 5 cost time: 135.7924289703369
Epoch: 1 cost time: 136.3309531211853
Epoch: 2 cost time: 136.66787457466125
Epoch: 3 cost time: 135.9030213356018
Epoch: 4 cost time: 135.05666089057922
Epoch: 5 cost time: 135.58430004119873
Epoch: 1 cost time: 135.03552341461182
Epoch: 2 cost time: 134.94299745559692
Epoch: 3 cost time: 135.6214325428009
Epoch: 4 cost time: 136.4595193862915
Epoch: 5 cost time: 136.3086621761322
Epoch: 1 cost time: 135.4370334148

Progress:   3%|▎         | 1/30 [1:29:57<43:28:42, 5397.33s/it]

Epoch: 1 cost time: 137.59531784057617
Epoch: 2 cost time: 136.6216335296631
Epoch: 3 cost time: 137.9029402732849
Epoch: 4 cost time: 137.82961106300354
Epoch: 5 cost time: 137.72088289260864
Epoch: 1 cost time: 137.00745964050293
Epoch: 2 cost time: 136.1413516998291
Epoch: 3 cost time: 136.09900879859924
Epoch: 4 cost time: 136.34603762626648
Epoch: 5 cost time: 136.98528861999512
Epoch: 1 cost time: 136.57419228553772
Epoch: 2 cost time: 136.97417664527893
Epoch: 3 cost time: 135.83490991592407
Epoch: 4 cost time: 137.0542380809784
Epoch: 5 cost time: 136.77235412597656
Epoch: 1 cost time: 136.59003901481628
Epoch: 2 cost time: 136.77531671524048
Epoch: 3 cost time: 136.7508463859558
Epoch: 4 cost time: 136.71640300750732
Epoch: 5 cost time: 136.50498414039612
Epoch: 1 cost time: 136.90982365608215
Epoch: 2 cost time: 136.37168049812317
Epoch: 3 cost time: 138.72238731384277
Epoch: 4 cost time: 137.11007833480835
Epoch: 5 cost time: 137.64273118972778
Epoch: 1 cost time: 135.492284

Progress:   7%|▋         | 2/30 [3:01:00<42:16:45, 5435.92s/it]

...

Progress:  93%|█████████▎| 28/30 [31:09:18<1:52:38, 3379.26s/it]

Epoch: 1 cost time: 74.20133423805237
Epoch: 2 cost time: 73.89994406700134
Epoch: 3 cost time: 73.89809584617615
Epoch: 4 cost time: 72.59024357795715
Epoch: 5 cost time: 73.12586402893066
Epoch: 1 cost time: 74.01140332221985
Epoch: 2 cost time: 72.0617003440857
Epoch: 3 cost time: 74.10417795181274
Epoch: 4 cost time: 73.49519610404968
Epoch: 5 cost time: 74.29171442985535
Epoch: 1 cost time: 73.30468201637268
Epoch: 2 cost time: 73.46861577033997
Epoch: 3 cost time: 73.06739139556885
Epoch: 4 cost time: 74.59945130348206
Epoch: 5 cost time: 75.31561231613159
Epoch: 1 cost time: 74.28547286987305
Epoch: 2 cost time: 74.7268476486206
Epoch: 3 cost time: 73.86970257759094
Epoch: 4 cost time: 74.8562068939209
Epoch: 5 cost time: 72.49697732925415
Epoch: 1 cost time: 73.45925641059875
Epoch: 2 cost time: 73.66653895378113
Epoch: 3 cost time: 73.10906648635864
Epoch: 4 cost time: 73.49177598953247
Epoch: 5 cost time: 73.39975786209106
Epoch: 1 cost time: 74.14498448371887
Epoch: 2 cost t

Progress:  97%|█████████▋| 29/30 [31:58:26<54:09, 3249.75s/it]  

Epoch: 1 cost time: 74.0227632522583
Epoch: 2 cost time: 74.33782768249512
Epoch: 3 cost time: 73.63359904289246
Epoch: 4 cost time: 73.71912455558777
Epoch: 5 cost time: 74.29706025123596
Epoch: 1 cost time: 74.26878547668457
Epoch: 2 cost time: 73.00489592552185
Epoch: 3 cost time: 73.38037157058716
Epoch: 4 cost time: 73.21849822998047
Epoch: 5 cost time: 73.78935885429382
Epoch: 1 cost time: 73.83876156806946
Epoch: 2 cost time: 73.72785091400146
Epoch: 3 cost time: 73.88761353492737
Epoch: 4 cost time: 74.51922726631165
Epoch: 5 cost time: 73.78617811203003
Epoch: 1 cost time: 73.04355120658875
Epoch: 2 cost time: 74.27742958068848
Epoch: 3 cost time: 73.06184506416321
Epoch: 4 cost time: 73.8922507762909
Epoch: 5 cost time: 73.65659642219543
Epoch: 1 cost time: 73.97974848747253
Epoch: 2 cost time: 73.07738780975342
Epoch: 3 cost time: 73.27068591117859
Epoch: 4 cost time: 72.95432877540588
Epoch: 5 cost time: 73.80373907089233
Epoch: 1 cost time: 73.26230955123901
Epoch: 2 cost 

Progress: 100%|██████████| 30/30 [32:47:32<00:00, 3935.08s/it]
